<a href="https://colab.research.google.com/github/jon-pun/Spring-2023-ORIE-4741/blob/main/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ORIE 4580/5580/5581 Assignment 4

#### Students: Jonathan Yun (jly37)

### Github link: [example Github link](https://github.com/SidBanerjee/ORIE4580-Simulation/blob/main/Notebooks/Unit0-Intro_Demos.ipynb)


### Instructions


* Due Friday September 29, at 11.59am on Gradescope.
* Assignment .ipynb files available for download on [Canvas](https://canvas.cornell.edu/courses/56826#:~:text=Assignments-,Assignments,-Assignments%20Module%20publish). Do all your work in provided notebook (text answers typeset in markdown; show all required code and generate plots inline), and then generate and submit a pdf.
* Ideally do assignments in groups of 2, and submit a single pdf with both names
* Please show your work and clearly mark your answers.
* You can use any code fragments given in class, found online (for example, on StackOverflow), or generated via Bard or ChatGPT (you are encouraged to use these for first drafts) **with proper referencing**. You can also discuss with others (again, please reference them if you do so); but you must write your final answers on your own as a team.



### Suggested reading

Chapters 5 (sections 5.1, 5.2 and 5.3) and 6, as well as chapter 11 (to revise again what we discussed in class about confidence intervals) of [Simulation by Ross](https://catalog.library.cornell.edu/catalog/12745977).




---
---


# Question 1: Multivariate normals, and some finance (20 points)


Assume the following means and covariance matrix for the net annual returns (annual returns after accounting for inflation) on three investments:

$$\boldsymbol{\mu}=
\begin{bmatrix}
    \mu_{\text{U.S. stocks}}\\
    \mu_{\text{U.S. bonds}}\\
    \mu_{\text{EM bonds}}
\end{bmatrix}=
\begin{bmatrix}
    0.14 \\
    0.07 \\
    0.16\\
\end{bmatrix},$$

Moreover, we have $\sigma^2_{\text{U.S. stocks}}=0.0625, \sigma^2_{\text{U.S. bonds}}=0.01$ and $\sigma^2_{\text{EM bonds}}=0.09$. Assuming all three instruments are independent of each other, what is the probability that all three financial products simultaneously yield a positive return?


___
## Ans.

Since US stocks, US bonds, and EM bonds are independent, the probability that all 3 products simultaneously yield a positive return is $P(\text{U.S. stocks} > 0) \times P(\text{U.S. bonds} > 0) \times P(\text{EM bonds} > 0)$, and each of these is simply based off the respective z-scores.

Here, find the following z-scores:

$$z_{\text{U.S. stocks}} = \frac{0 - 0.14}{\sqrt{.0625}} = -0.56$$
$$z_{\text{U.S. bonds}} = \frac{0 - 0.17}{\sqrt{.01}} = -0.538$$
$$z_{\text{EM bonds}} = \frac{0 - 0.16}{\sqrt{.09}} = -0.533$$

Thus, we find the probability that all 3 products simultaneously yield a positive return is $P(Z < -0.56) \times P(Z < -0.538) \times P(Z < -0.533)$, which we can calculate using Python:


In [1]:
import scipy as sc
from scipy import stats

prob_us_stock = stats.norm.cdf(-.56, loc = 0, scale = 1)
prob_us_bond = stats.norm.cdf(-.538, loc = 0, scale = 1)
prob_em_bond = stats.norm.cdf(-.533, loc = 0, scale = 1)

total_prob = prob_us_stock * prob_us_bond * prob_em_bond
print(total_prob)

0.025236398973361475


___

**(b)** Next, assume that the assets are not independent, but rather, have covariance matrix
$$\boldsymbol{\Sigma}=
\begin{bmatrix}
    0.0625 & -0.01 & 0.0375 \\
    -0.01 & 0.01 & -0.015 \\
    0.0375 & -0.015 & 0.09
\end{bmatrix}.$$

Using simulation, estimate the probability that all three financial products simultaneously yield a positive return.
For the purposes of this problem, you are **not allowed** to use the Python function ${\tt numpy.random.multivariate\_normal}$, but rather, should generate *independent* normal rvs, and then apply the method(s) discussed in class.
In particular, you can compute the Cholesky decomposition of the covariance matrix using the Python function ${\tt scipy.linalg.cholesky(sigma, lower=True)}$.
___

## Ans.



In [2]:
import numpy as np
from scipy import linalg

In [3]:
results = []

for i in range(5000):

  sigma = np.array([[.0625, -.01, .0375],
                    [-.01, .01, -0.015],
                    [.0375, -.015, 0.09]])

  L = linalg.cholesky(sigma, lower=True)

  X = np.random.normal(0,1, 3).reshape((3,1))

  Y = L @ X + np.array([[.14, .07, .16]])

  sum = np.sum(Y)

  results.append(int(sum < 0))

print(np.sum(results) / 5000)

0.1838


___
___

# Question 2: Covariance and copulas (30 points)

In general (i.e., apart from multivariate normals), *there is no closed form formula for generating rvs with given marginals and given covariance matrix*. However, a popular heuristic is the so-called *Gaussian copula trick*, which is widely used in finance. In this question, we will try and get a sense of how we can generate correlated uniforms, and how the Gaussian copula trick works.

___

**(a)** We will first try and make a method for generating two correlated uniform random variables $V_1$, $V_2$ with mean $\mathbb{E}[V_1]=\mathbb{E}[V_2]=0$, variance $Var(V_1)=Var(V_2)=1$, and covariance $Cov( V_1, V_2 ) = \sigma_{12}$ (for some given $\sigma_{12}\geq 0$).

For what $\alpha,\beta\in\mathbb{R}$ does a uniform distribution on $[\alpha,\beta]$ have mean $0$ and variance $1$.
___


## Ans.


___

**(b)** Next, generate rvs $X_1,X_2\sim Uniform[\alpha, \beta]$ i.i.d (with $\alpha,\beta$ from previous part), and also sample $B\sim Bernoulli(p)$. Define:
\begin{align*}
	V_1 = X_1 \qquad,\qquad
	V_2 = (1-B)X_1+BX_2 = \begin{cases} X_1 & \text{ if $B = 0$} \\ X_2 & \text{ if $B = 1$} \end{cases}
	\end{align*}

What is $\mathbb{E}[V_2]$ and $Var(V_2)$?
___

## Ans.


___
**(c)** For what value of $p$ do we get $Cov(V_1, V_2 )=\sigma_{12}$ (for $\sigma_{12}\geq 0$)?
___

## Ans.


___

**(d)** Create a function to generate two correlated uniform random variables $V_1$, $V_2$ such that $\mathbb{E}[V_1]=\mathbb{E}[V_2]=0,Var(V_1)=Var(V_2)=1$ and $Cov( U_1, U_2 ) = \sigma_{12}$. Use this to show a scatter plot of $(V_1,V_2)$ for correlation values $\sigma_{12}=0,0.25,0.5,0.75$ and $1$.

___

## Ans.

___

**(e)** Using the above function, compute $\mathbb{P}[V_1\leq 0,V_2\leq 0]$ for the case when $\sigma_{12}=0.5$ (in a financial application, this can correspond to *both* $V_1$ and $V_2$ 'defaulting').
___

## Ans.

___

**(f)** The Gaussian copula trick is based on first generating a multivariate normal vector with given covariance matrix $\Sigma$, and then using inversion to convert it to a vector of uniform rvs, which can then used to generate the desired rvs. We now demonstrate this in the above setting.

Let $\phi(\cdot)$ denote the 1-d normal cdf. Generate $(Z_1,Z_2)$ as multivariate normals with covariance
$\Sigma =\begin{pmatrix}
		1 & 0.5 \\
		0.5 & 1
	\end{pmatrix}$,
and define $W_1 = \alpha + (\beta-\alpha)\phi(Z_1)$ and $W_2 = \alpha + (\beta-\alpha)\phi(Z_2)$.

Argue $W_1$, $W_2$ are uniform on $[\alpha,\beta]$ (and hence $\mathbb{E}[W_1]=\mathbb{E}[W_2]=0,Var(W_1)=Var(W_2)=1$). Next, for $\sigma_{12}=0.5$ and $\alpha,\beta$ as above, write a function to generate $W_1,W_2$ as described, and use these to plot the scatter plot, and compute $\mathbb{P}[W_1\geq 0,W_2\geq 0]$.

___

## Ans.


___
___

# Question 3: Generating random samples in some given set (20 pts)

___

**(a)** We want to sample points which are uniformly distributed in the set  $\mathcal{S} = \{(x,y)| x\in \mathbb{R}, y \in [0,e^{-x^2/2}]\}$ (i.e., between the lines $y=0$ and $y=e^{-x^2/2}$). Generate 1000 independent samples $(X_i,Y_i)$, and plot them using a scatterplot, along with the lines $y=0$ and $y=e^{-x^2/2}$.

(Hint: Try to 'reverse' what we do in Acceptance-Rejection).
___

## Ans.

___
**(b)** Next, suppose we want to sample points which are uniformly distributed on the set $\{(x,y)\in\mathbb{R}^2|x^2+y^2 = 1\}$ (i.e., on the *circumference* of the unit circle). One way to do so may be to generate points $(U_i,V_i)$ uniformly in $[-1,1]^2$ (like in the first homework), and then *project* them to the circle, i.e., take $(X_i,Y_i) = \left(\frac{U_i}{\sqrt{U_i^2+V_i^2}},\frac{V_i}{\sqrt{U_i^2+V_i^2}}\right)$. Generate $1000$ points in this way, and plot them using a scatter plot.

Are these uniformly distributed on the circumference of the circle? Try checking the histogram of their angles (i.e., the histogram of $\theta_i = \tan^{-1}(Y_i/X_i)$).
___

## Ans.



---

**(c)** Next, generate points $(Z_i,W_i)$ as independent standard normals (i.e., $(Z_i,W_i)\sim \mathcal{N}(0,I_2)$, and again project them to the circle, i.e., take $(X_i,Y_i) = \left(\frac{Z_i}{\sqrt{Z_i^2+W_i^2}},\frac{W_i}{\sqrt{Z_i^2+W_i^2}}\right)$. Generate $1000$ points in this way, and plot them using a scatter plot.

Are these uniformly distributed on the circumference of the circle? Again, check the histogram of their angles $\theta_i$. Can you explain the difference between parts (b) and (c)?

___

## Ans.

___
___

# Question 4: The Inspection paradox (30 pts)

Have you ever felt like you wait too long for elevators, taxis, buses, etc.? Guess what - you may be right!

We are going to simulate a number of train and passenger arrivals at a station, and try to measure the time each passenger needs to wait for a train. To do this, first generate a stream of 10,000 train passengers, each of whom have an independent arrival time which is uniformly distributed over an interval of 1,000 minutes (i.e., passenger $i$ arrives at time $T_i\sim [0,1,000]$. Sort the passengers by arrival time, and use this resulting vector for all subsequent parts.

___

**(a)** Suppose trains arrive at the station deterministically every 10 minutes. Assume each train has infinite capacity, so that everyone waiting at the station when a train arrives can get on; also assume that passengers get on the train instantaneously. Estimate the expected time (with CIs) that any *randomly chosen* passenger spends waiting at the station for the next train.
___


## Ans.

In [4]:
arrivals = np.random.uniform(0, 1000, 10000)
arrivals = np.sort(arrivals)

In [20]:
train_arrivals = np.arange(start = 0, stop = 1010, step = 10)
wait_time = np.zeros(10000)

for i in range(len(arrivals)):
  time_diff = train_arrivals - arrivals[i]
  min_positive_time = time_diff[np.where(time_diff > 0, time_diff, np.inf).argmin()]
  wait_time[i] = min_positive_time

print(wait_time)

mean_wait = np.mean(wait_time)
sd_wait = np.std(wait_time)



print("mean wait time: {}".format(mean_wait))
print("95% CI lower bound: {}".format(mean_wait - 1.96 * (sd_wait / np.sqrt(10000))))
print("95% CI upper bound: {}".format(mean_wait + 1.96 * (sd_wait / np.sqrt(10000))))


[9.8879385  9.72902642 9.60813499 ... 0.31717642 0.19101892 0.14525586]
mean wait time: 4.976160907168059
95% CI lower bound: 4.919766632225652
95% CI upper bound: 5.032555182110466


___
**(b)** Next, assume trains arrive at the station with *inter-arrival* time (i.e., time between two trains) distributed according to an exponential distribution with mean 10 minutes. Again, use your vector of passenger arrival times to estimate the expected waiting time for a randomly chosen passenger.
___


## Ans.

In [21]:
train_arrivals = []
while(np.sum(train_arrivals) < 1000):
  train_arrivals.append(np.random.exponential(scale = 10))

train_arrivals = np.array(train_arrivals)
c_train_arrivals = np.cumsum(train_arrivals)

wait_time = np.zeros(10000)

for i in range(len(arrivals)):
  time_diff = c_train_arrivals - arrivals[i]
  min_positive_time = time_diff[np.where(time_diff > 0, time_diff, np.inf).argmin()]
  wait_time[i] = min_positive_time

print(wait_time)

mean_wait = np.mean(wait_time)
sd_wait = np.std(wait_time)



print("mean wait time: {}".format(mean_wait))
print("95% CI lower bound: {}".format(mean_wait - 1.96 * (sd_wait / np.sqrt(10000))))
print("95% CI upper bound: {}".format(mean_wait + 1.96 * (sd_wait / np.sqrt(10000))))

[ 8.78314812  8.62423603  8.5033446  ... 18.60761416 18.48145666
 18.4356936 ]
mean wait time: 9.29128370143432
95% CI lower bound: 9.129916721720885
95% CI upper bound: 9.452650681147755


___

**(c)** (Hopefully, you found the answer in part (b) to be different, and surprisingly so, compared to part (a).)

The train service claims that there is a train every $10$ minutes on average. In part (a), this is clearly the case, but is it so in part (b) as well? To understand this, use your data from part (b) to compute an empirical estimate with CIs for the inter-arrival times between trains.

___

In [23]:
mean_train_arrivals = np.mean(train_arrivals)
sd_train_arrivals = np.std(train_arrivals)

print("mean train arrival time: {}".format(mean_train_arrivals))
print("95% CI lower bound: {}".format(mean_train_arrivals - 1.96 * (sd_train_arrivals / np.sqrt(len(train_arrivals)))))
print("95% CI upper bound: {}".format(mean_train_arrivals + 1.96 * (sd_train_arrivals / np.sqrt(len(train_arrivals)))))

100
mean train arrival time: 10.182904377395662
95% CI lower bound: 8.376067207274403
95% CI upper bound: 11.989741547516921


In fact, train arrivals are still within the bound of "every 10 minutes on average"; the CI contains 10 minutes.

___

**(d)** To understand what may be happening between parts (a) and (b), use the data from the simulations in parts (a) and (b) to estimate the average inter-arrival time between trains that each passenger experiences in each case. In other words, if passenger $i$ gets on train number $n$, thn we want to define his inter-arrival time $I_i$ to be the difference $A_n~-~A_{n-1}$ where $A_n$ is the arrival time of train $n$.

Compute the sample mean and CI of the interval $I_i$ for a randomly chosen passenger $i$ in each of the two settings. How are these related to the waiting time estimates you got in each part? Can you explain this?

___

## Ans.

In [24]:
# for part (a), with arrival times every 10 minutes

train_arrivals = np.arange(start = 0, stop = 1010, step = 10)
wait_time = np.zeros(10000)

for i in range(len(arrivals)):
  time_diff = train_arrivals - arrivals[i]
  min_positive_time = train_arrivals[np.where(time_diff > 0, time_diff, np.inf).argmin()] - train_arrivals[np.where(time_diff > 0, time_diff, np.inf).argmin() - 1]
  wait_time[i] = min_positive_time

print(wait_time)

mean_wait = np.mean(wait_time)
sd_wait = np.std(wait_time)



print("mean wait time: {}".format(mean_wait))
print("95% CI lower bound: {}".format(mean_wait - 1.96 * (sd_wait / np.sqrt(10000))))
print("95% CI upper bound: {}".format(mean_wait + 1.96 * (sd_wait / np.sqrt(10000))))

[10. 10. 10. ... 10. 10. 10.]
mean wait time: 10.0
95% CI lower bound: 10.0
95% CI upper bound: 10.0


In [26]:
# for part (b), with arrival times exponentially distributed with mean 10 minutes

train_arrivals = [0]
while(np.sum(train_arrivals) < 1000):
  train_arrivals.append(np.random.exponential(scale = 10))

train_arrivals = np.array(train_arrivals)
c_train_arrivals = np.cumsum(train_arrivals)

wait_time = np.zeros(10000)

for i in range(len(arrivals)):
  time_diff = c_train_arrivals - arrivals[i]
  min_positive_time = c_train_arrivals[np.where(time_diff > 0, time_diff, np.inf).argmin()] - c_train_arrivals[np.where(time_diff > 0, time_diff, np.inf).argmin() - 1]
  wait_time[i] = min_positive_time

print(wait_time)

mean_wait = np.mean(wait_time)
sd_wait = np.std(wait_time)



print("mean wait time: {}".format(mean_wait))
print("95% CI lower bound: {}".format(mean_wait - 1.96 * (sd_wait / np.sqrt(10000))))
print("95% CI upper bound: {}".format(mean_wait + 1.96 * (sd_wait / np.sqrt(10000))))

[19.70176028 19.70176028 19.70176028 ... 32.33446006 32.33446006
 32.33446006]
mean wait time: 17.75573847588684
95% CI lower bound: 17.56129013071676
95% CI upper bound: 17.95018682105692


The average waiting time is, in essence, half of the average time between arrivals. This makes sense: for a uniform distribution, we'd calculate the average wait time for the "group" of passengers all getting onto one train as half of the inter-arrival time, which then compounds.

**(e) (Optional for all students)** Based on your previous parts, can you come up with an explanation for what is going on?  

Yes! In essence, in the exponential case, since there are trains with an inter-arrival time > 10 minutes, they "collect" more passengers waiting at the station, and thus the average wait time is pushed up by these cases (since the average wait time, as noted in (d), would be > 5 minutes). Since many more of these values are recorded (compared to if the inter-arrival time were < 10 minutes, which fewer passengers board, and thus fewer average waits of < 5 minutes are recorded), this skews the mean wait time up.